In [2]:
import json
import pickle
import numpy as np
import matplotlib.pyplot as plt
from scipy.odr import RealData, ODR, Model as ODRModel
import logging

# Set up basic logging to see the output
logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')

def read_json_data(json_path):
    """
    Reads scintillation data from a pipeline-generated JSON file.

    Args:
        json_path (str): The path to the input JSON file.

    Returns:
        tuple: A tuple containing arrays for frequencies (MHz), bandwidths (MHz),
               bandwidth errors (MHz), modulation indices, and modulation index errors.
               Returns (None, None, None, None, None) if an error occurs.
    """
    logging.info(f"Reading data from JSON file: {json_path}")
    try:
        with open(json_path, 'r') as f:
            data = json.load(f)

        # Navigate to the sub-band measurements. We assume the main component is 'scint_scale'.
        # This might need to be adjusted if the component name changes.
        measurements = data['components']['scint_scale']['subband_measurements']

        freqs = np.array([m['freq_mhz'] for m in measurements])
        bws = np.array([m['bw'] for m in measurements])
        mods = np.array([m['mod'] for m in measurements])

        # Combine statistical fit error and finite scintle error in quadrature
        bw_err_fit = np.nan_to_num(np.array([m.get('bw_err') for m in measurements]))
        bw_err_finite = np.nan_to_num(np.array([m.get('finite_err') for m in measurements]))
        bw_errs = np.sqrt(bw_err_fit**2 + bw_err_finite**2)
        
        mod_errs = np.nan_to_num(np.array([m.get('mod_err') for m in measurements]))

        return freqs, bws, bw_errs, mods, mod_errs

    except (FileNotFoundError, KeyError, TypeError) as e:
        logging.error(f"Could not read or parse JSON file {json_path}. Error: {e}")
        return None, None, None, None, None

def read_pkl_data(pkl_path, model_key='1_lorenz'):
    """
    Reads scintillation data from a pickle file containing analysis results.

    Args:
        pkl_path (str): The path to the input pickle file.
        model_key (str): The key for the desired model fit ('1_lorenz' or '2_lorenz').
                         Defaults to '1_lorenz' for a single scintillation scale.

    Returns:
        tuple: A tuple containing arrays for frequencies (MHz), bandwidths (MHz),
               bandwidth errors (MHz), modulation indices, and modulation index errors.
               Returns (None, None, None, None, None) if an error occurs.
    """
    logging.info(f"Reading data from pickle file: {pkl_path}")
    try:
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        #print(data)

        # Extract data based on the chosen model key
        if model_key not in data:
            logging.error(f"Model key '{model_key}' not found in pickle file.")
            return None, None, None, None, None
            
        freqs = np.array(data['f_cents'])
        
        # For a 1-component model, the keys are straightforward
        if model_key == '1_lorenz':
            bws = np.array(data[model_key]['sub_scint_1'])
            mods = np.array(data[model_key]['mods1'])
            bw_err_fit = np.nan_to_num(np.array(data[model_key]['sub_scint_uncert_1']))
            mod_errs = np.nan_to_num(np.array(data[model_key]['mods1_uncert']))
            
            # The 'add_un1' seems to be an additional uncertainty. We'll take its mean value
            # for each sub-band and add it in quadrature to the fit error.
            bw_err_add = np.array([np.mean(err) for err in data[model_key]['add_un1']])
            bw_errs = np.sqrt(bw_err_fit**2 + bw_err_add**2)
            
            return freqs, bws, bw_errs, mods, mod_errs

        # For a 2-component model, we must choose which component to use.
        # Here, we default to the broader component (sub_scint_2), a common choice.
        elif model_key == '2_lorenz':
            bws1 = np.array(data[model_key]['sub_scint_1'])
            bws2 = np.array(data[model_key]['sub_scint_2']) # Choosing the second component
            mods1 = np.array(data[model_key]['mods1'])
            mods2 = np.array(data[model_key]['mods2'])
            bw_err_fit1 = np.nan_to_num(np.array(data[model_key]['sub_scint_uncert_1']))
            bw_err_fit2 = np.nan_to_num(np.array(data[model_key]['sub_scint_uncert_2']))
            mod_errs1 = np.nan_to_num(np.array(data[model_key]['mods1_uncert']))
            mod_errs2 = np.nan_to_num(np.array(data[model_key]['mods2_uncert']))
            bw_err_add1 = np.array([np.mean(err) for err in data[model_key]['add_un1']])
            bw_err_add2 = np.array([np.mean(err) for err in data[model_key]['add_un2']])
            bw_errs1 = np.sqrt(bw_err_fit1**2 + bw_err_add1**2)
            bw_errs2 = np.sqrt(bw_err_fit2**2 + bw_err_add2**2)
            
            return freqs, bws1, bws2, bw_errs1, bw_errs2, mods1, mods2, mod_errs1, mod_errs2
        
        else:
            raise ValueError(f"Unsupported model key: {model_key}")
            return None, None, None, None, None
                
            


        

    except (FileNotFoundError, KeyError, TypeError, pickle.UnpicklingError) as e:
        logging.error(f"Could not read or parse pickle file {pkl_path}. Error: {e}")
        return None, None, None, None, None




In [3]:
mod_key = '1_lorenz'
out = read_pkl_data('wilhelm_253635173_subband_acf_fits.pkl', model_key=mod_key)
print('Wilhelm')
print('______')
print(f'Model: {mod_key}')
print(f'freqs: {out[0]}')
print(f'bws: {out[1]}')
print(f'bw_errs: {out[2]}')
print(f'mods: {out[3]}')
print(f'mod_errs: {out[4]}')
print('\n')
mod_key = '2_lorenz'
out = read_pkl_data('wilhelm_253635173_subband_acf_fits.pkl', model_key=mod_key)
print('Wilhelm')
print('______')
print(f'Model: {mod_key}')
print('Comp. 1')
print('______')
print(f'freqs: {out[0]}')
print(f'bws: {out[1]}')
print(f'bw_errs: {out[3]}')
print(f'mods: {out[5]}')
print(f'mod_errs: {out[7]}')
print('Comp. 2')
print('______')
print(f'freqs: {out[0]}')
print(f'bws: {out[2]}')
print(f'bw_errs: {out[4]}')
print(f'mods: {out[6]}')
print(f'mod_errs: {out[8]}')

INFO: Reading data from pickle file: wilhelm_253635173_subband_acf_fits.pkl
INFO: Reading data from pickle file: wilhelm_253635173_subband_acf_fits.pkl


Wilhelm
______
Model: 1_lorenz
freqs: [727.9774772  693.3742263  674.49903223 639.57916025]
bws: [0.23917776 0.03098038 0.08902349 0.03060903]
bw_errs: [0.17173404 0.01466097 0.06176402 0.01832361]
mods: [0.30050298 0.23880883 0.21708979 0.28788154]
mod_errs: [0.09866258 0.03351959 0.03441208 0.04568383]


Wilhelm
______
Model: 2_lorenz
Comp. 1
______
freqs: [727.9774772  693.3742263  674.49903223 639.57916025]
bws: [0.00307365 0.02570941 0.08895985 0.03059751]
bw_errs: [0.02494494 0.02007905 0.00673388 0.02739737]
mods: [0.18223351 0.22781298 0.12695976 0.28786561]
mod_errs: [0.88436624 0.04374701 0.         0.06390525]
Comp. 2
______
freqs: [727.9774772  693.3742263  674.49903223 639.57916025]
bws: [0.25169947 4.26647844 0.08898911 8.03735346]
bw_errs: [2.10470173e-01 2.03455627e+03 6.73717548e-03 1.54661591e+07]
mods: [0.30295555 1.63156369 0.17608887 0.13642623]
mod_errs: [1.15612534e-01 7.74038459e+02 0.00000000e+00 2.62065645e+05]


In [4]:
mod_key = '1_lorenz'
out = read_pkl_data('hamilton_318353610_subband_acf_fits.pkl', model_key=mod_key)
print('Hamilton')
print('______')
print(f'Model: {mod_key}')
print(f'freqs: {out[0]}')
print(f'bws: {out[1]}')
print(f'bw_errs: {out[2]}')
print(f'mods: {out[3]}')
print(f'mod_errs: {out[4]}')
print('\n')
mod_key = '2_lorenz'
out = read_pkl_data('hamilton_318353610_subband_acf_fits.pkl', model_key=mod_key)
print('Hamilton')
print('______')
print(f'Model: {mod_key}')
print('Comp. 1')
print('______')
print(f'freqs: {out[0]}')
print(f'bws: {out[1]}')
print(f'bw_errs: {out[3]}')
print(f'mods: {out[5]}')
print(f'mod_errs: {out[7]}')
print('Comp. 2')
print('______')
print(f'freqs: {out[0]}')
print(f'bws: {out[2]}')
print(f'bw_errs: {out[4]}')
print(f'mods: {out[6]}')
print(f'mod_errs: {out[8]}')

INFO: Reading data from pickle file: hamilton_318353610_subband_acf_fits.pkl
INFO: Reading data from pickle file: hamilton_318353610_subband_acf_fits.pkl


Hamilton
______
Model: 1_lorenz
freqs: [751.89172132 692.02950923 669.6767386  637.74137532 601.92872416
 562.99257505 507.81789873 438.82017994]
bws: [0.02842666 0.02473602 0.02577682 0.02044736 0.02446064 0.0223271
 0.00637338 0.06494488]
bw_errs: [0.00343674 0.00549508 0.00830597 0.00547515 0.00621152 0.02139245
 0.00442823 0.04313377]
mods: [0.59461069 0.36450813 0.31072536 0.37388121 0.37165514 0.21786325
 0.60190901 0.35752295]
mod_errs: [0.01967348 0.02913962 0.03581789 0.03771235 0.03371395 0.07755643
 0.25108882 0.05352274]


Hamilton
______
Model: 2_lorenz
Comp. 1
______
freqs: [751.89172132 692.02950923 669.6767386  637.74137532 601.92872416
 562.99257505 507.81789873 438.82017994]
bws: [2.84266560e-02 2.40408093e-02 2.57781100e-02 1.54204163e-02
 2.44592669e-02 6.46429413e-05 6.40043519e-03 6.49239495e-02]
bw_errs: [0.00383876 0.93125324 0.00930369 0.00553507 0.00722262 1.2116245
 0.00465558 0.1533518 ]
mods: [ 0.59461072  0.33590128  0.31072123  0.38164009  0.37166081 22.9

In [5]:
mod_key = '1_lorenz'
out = read_pkl_data('chromatica_356959136_subband_acf_fits.pkl', model_key=mod_key)
print('Chromatica')
print('______')
print(f'Model: {mod_key}')
print(f'freqs: {out[0]}')
print(f'bws: {out[1]}')
print(f'bw_errs: {out[2]}')
print(f'mods: {out[3]}')
print(f'mod_errs: {out[4]}')
print('\n')
mod_key = '2_lorenz'
out = read_pkl_data('chromatica_356959136_subband_acf_fits.pkl', model_key=mod_key)
print('Chromatica')
print('______')
print(f'Model: {mod_key}')
print('Comp. 1')
print('______')
print(f'freqs: {out[0]}')
print(f'bws: {out[1]}')
print(f'bw_errs: {out[3]}')
print(f'mods: {out[5]}')
print(f'mod_errs: {out[7]}')
print('Comp. 2')
print('______')
print(f'freqs: {out[0]}')
print(f'bws: {out[2]}')
print(f'bw_errs: {out[4]}')
print(f'mods: {out[6]}')
print(f'mod_errs: {out[8]}')

INFO: Reading data from pickle file: chromatica_356959136_subband_acf_fits.pkl
INFO: Reading data from pickle file: chromatica_356959136_subband_acf_fits.pkl


Chromatica
______
Model: 1_lorenz
freqs: [775.19521713 725.1950264  675.19483566 625.19464493 575.19445419
 525.19426345 475.19407272 425.19464493]
bws: [0.15512703 0.09312848 0.08201245 0.06500323 0.04325292 0.03595521
 0.04195829 9.74846035]
bw_errs: [1.37423570e-02 7.02233758e-03 5.06578814e-03 4.08970916e-03
 2.99665509e-03 3.60029574e-03 7.63257705e-03 4.63948410e+03]
mods: [0.82407898 0.57196486 0.61297166 0.53116217 0.47741359 0.45226468
 0.41587799 4.41339781]
mod_errs: [6.71719175e-03 5.37741968e-03 2.36299792e-03 4.51353888e-03
 6.96967022e-03 1.16595446e-02 2.05510469e-02 2.09591219e+03]


Chromatica
______
Model: 2_lorenz
Comp. 1
______
freqs: [775.19521713 725.1950264  675.19483566 625.19464493 575.19445419
 525.19426345 475.19407272 425.19464493]
bws: [0.15511829 0.09074251 0.08201235 0.06500297 0.04325085 0.03595325
 0.04081399 0.05808479]
bw_errs: [1.23315462e+02 9.62148424e-03 5.22947340e-03 4.62825607e-03
 3.73983078e-03 4.74292482e-03 1.03819576e-02 8.57649684e-02]
m

In [6]:
mod_key = '1_lorenz'
out = read_pkl_data('freya_278720455_subband_acf_fits.pkl', model_key=mod_key)
print('Freya')
print('______')
print(f'Model: {mod_key}')
print(f'freqs: {out[0]}')
print(f'bws: {out[1]}')
print(f'bw_errs: {out[2]}')
print(f'mods: {out[3]}')
print(f'mod_errs: {out[4]}')
print('\n')
mod_key = '2_lorenz'
out = read_pkl_data('freya_278720455_subband_acf_fits.pkl', model_key=mod_key)
print('Freya')
print('______')
print(f'Model: {mod_key}')
print('Comp. 1')
print('______')
print(f'freqs: {out[0]}')
print(f'bws: {out[1]}')
print(f'bw_errs: {out[3]}')
print(f'mods: {out[5]}')
print(f'mod_errs: {out[7]}')
print('Comp. 2')
print('______')
print(f'freqs: {out[0]}')
print(f'bws: {out[2]}')
print(f'bw_errs: {out[4]}')
print(f'mods: {out[6]}')
print(f'mod_errs: {out[8]}')

INFO: Reading data from pickle file: freya_278720455_subband_acf_fits.pkl
INFO: Reading data from pickle file: freya_278720455_subband_acf_fits.pkl


Freya
______
Model: 1_lorenz
freqs: [763.56110903 707.96626194 670.4302606  553.05005209]
bws: [0.05846214 0.05657032 0.04009965 0.0325654 ]
bw_errs: [0.00971183 0.00836955 0.00947453 0.00374005]
mods: [0.18321074 0.20777054 0.17085297 0.37465671]
mod_errs: [0.00721333 0.00763965 0.01116422 0.0092641 ]


Freya
______
Model: 2_lorenz
Comp. 1
______
freqs: [763.56110903 707.96626194 670.4302606  553.05005209]
bws: [0.05846548 0.00244179 0.04010179 0.0298044 ]
bw_errs: [0.02125671 0.00360497 0.01400864 0.0052999 ]
mods: [0.18321008 0.29073929 0.1708517  0.36596963]
mod_errs: [0.0272018  0.27761561 0.01471213 0.0171679 ]
Comp. 2
______
freqs: [763.56110903 707.96626194 670.4302606  553.05005209]
bws: [0.90223133 0.07104792 0.28692445 5.13706809]
bw_errs: [6.69986273e+08 1.45853748e-02 9.46455569e+10 2.50060623e+03]
mods: [3.93907655e-05 1.92564847e-01 2.12082130e-08 2.17695427e+00]
mod_errs: [2.50765508e+04 1.07614323e-02 9.31868981e+04 1.05594054e+03]


In [21]:
# In an environment that has lmfit & uncertainties installed
import pickle, lmfit, uncertainties
with open("wilhelm_253635173_subband_acf_fits.pkl", "rb") as f:
    acf_dict = pickle.load(f)

#print(acf_dict.keys())  # likely: dict_keys(['sub_scint_1', 'sub_scint_2', ...])
#print(acf_dict['2_lorenz'])
print(acf_dict['f_cents'])
print(acf_dict['1_lorenz']['sub_scint_1'])    # lmfit.ModelResult or similar
print(acf_dict['1_lorenz']['sub_scint_uncert_1'])
#print(acf_dict['2_lorenz']['sub_scint_2'])

[727.977477196368, 693.3742262989571, 674.4990322254934, 639.5791602472219]
[0.239177755414001, 0.03098038269728807, 0.08902349045914636, 0.03060903481854996]
[0.16924869256962413, 0.014594476244811576, 0.06139504876062839, 0.01827234228783935]


In [3]:
!pip install lmfit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 34.0 MB/s eta 0:00:00
